# 01/2022 Questions

This notebook will track progress on practice questions that are sent by the InterviewQs website. 

In [118]:
# THIS FUNCTION TAKES A CSV FILE FROM A GITHUB URL AND READS IT INTO A PANDAS DATA FRAME
import pandas as pd

def read_file(url):

    """
    Takes GitHub url as an argument,
    pulls CSV file located @ github URL.

    """

    url = url + "?raw=true"
    df = pd.read_csv(url)
    return df

# READ FILE FROM GITHUB REPO
url_2022_01_17_01 = "https://github.com/akstl1/InterviewQs-Data-Science-Questions/blob/main/Data/2022.01.17-ad_table.csv"
url_2022_01_17_02 = "https://github.com/akstl1/InterviewQs-Data-Science-Questions/blob/main/Data/2022.01.17-spend_table.csv"
url_2022_01_14 = "https://github.com/akstl1/InterviewQs-Data-Science-Questions/blob/main/Data/2022.01.14_data.csv"
url_2022_01_21="https://github.com/akstl1/InterviewQs-Data-Science-Questions/blob/main/Data/2022.01.21_data.csv"
url_2022_01_26="https://github.com/akstl1/InterviewQs-Data-Science-Questions/blob/main/Data/2022.01.26-sf_salaries.csv"


ad_table_data= read_file(url_2022_01_17_01)
spend_table_data = read_file(url_2022_01_17_02)
channel_data = read_file(url_2022_01_14)
cereal_data = read_file(url_2022_01_21)
salary_data = read_file(url_2022_01_26)

C:\Users\allan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (3,4,5,6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


<hr style="border:1px solid black"> </hr>

## 1/26 Question

You have the following  dataset  containing salaries for public workers of San Francisco, CA. Let's assume this data is in a table called sf_salaries.

Can you write a SQL query to find the top 3 highest paid and top 3 lowest paid job titles?

|Column Name|	Column Type|	Short description|
|---|---|---|
|Id|	int|	The primary key of this table.|
|EmployeeName|	string|	The full name of the employee.|
|JobTitle|	string|	The title of the job the person does.|
|BasePay|	double|	The base amount of money the employee earns per year.|
|OvertimePay|	double|	The amount of money earned for the whole year for work done overtime.|
|OtherPay|	string|	Extra money coming from other activities. |
|Benefits|	string|	{incomplete data}|
|Totalpay|	double|	The total without benefits.|
|TotalPayBenefits|	double|	Total pay with benefits.|
|Year|	int|	The year when the money was paid out.
|Notes|	string|	{incomplete data}|
|Agency|	string|	Always is 'San Francisco'.|
|Status|	string|	{incomplete data}|

### Approach

Notes/Assumptions:
- First I will solve this problem in Python so each step can be seen in this Jupyter notebook, then I will post an equivalent SQL solution
- While exploring the data, I see that there are some entries with total pay of 0 or similarly low values. I will assume that these are people who either quit, were owed one check, or only worked a short period of time. I will exclude these results as they don't represent the true salary of an employee. For this lower end threshold, I will use $8000. This represents the approximate salary someone would earn at minimum wage (8/hour is 2011) working part time at 20 hours per week. Though this assumption itself could be modified, this seemed like a more representative lower end of the salary range in the work force.

To solve this problem I will:
- only keep the JobTitle, TotalPayBenefits cols since those will determine salary by job type
- group our data by JobTitle and average TotalPayBenefits by JobTitle accordingly
- find the top three and bottom three results in the above to get the highest and lowest paid titles

In [83]:
#import data
salary_df = salary_data

#cut off lower range of data as described above
salary_df=salary_df[salary_df["TotalPay"]>=8000]

In [84]:
#view data's head
salary_df.head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411,0,400184,NaN,567595.43,567595.43,2011,NaN,San Francisco,NaN
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966,245132,137811,NaN,538909.28,538909.28,2011,NaN,San Francisco,NaN
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739,106088,16452.6,NaN,335279.91,335279.91,2011,NaN,San Francisco,NaN
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916,56120.7,198307,NaN,332343.61,332343.61,2011,NaN,San Francisco,NaN
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134402,9737,182235,NaN,326373.19,326373.19,2011,NaN,San Francisco,NaN


In [72]:
#drop irrelevant cols
salary_df=salary_df.drop(["Id","EmployeeName","Year","Notes","Agency","Status", "TotalPay"], axis=1)

In [74]:
#group data by title, aggregate pay/benefits as an average, and sort by pay/benefits
average_salary_df = salary_df.groupby("JobTitle").agg("mean").sort_values(by="TotalPayBenefits")

In [81]:
#extract and show top pay/benefit titles
top_three_salaries = average_salary_df[-3:]
top_three_salaries

,TotalPayBenefits
JobTitle,
"Chief, Fire Department",408865.326667
Chief of Police,411732.266667
Chief Investment Officer,436224.360000


In [82]:
#extract and show bottom pay/benefit titles
bottom_three_salaries = average_salary_df[:3]
bottom_three_salaries

,TotalPayBenefits
JobTitle,
Camp Assistant,9255.350000
SCHOOL CROSSING GUARD,9267.573929
CITY PLANNING INTERN,9433.090000


### Results

As shown above, the positions with top Total Pay and Benefits were FD Chief, Chief of Police, and CIO; the positions with the lowest total pay and benefits were Camp assistant, school crossing guard and city planning intern.

### SQL Query to match the above

SELECT TOP(3) * from (SELECT JobTitle, average(TotalPayBenefits) from salary_df
WHERE TotalPayBenefits>8000)
UNION
SELECT BOTTOM(3) * from (SELECT BOTTOM(3) JobTitle, average(TotalPayBenefits) from salary_df
where TotalPayBenefits>8000)

#### Further Work

To achieve a better result, with more work/data, the following could be done/explored:
- Reading through all rows to weed out rows with lower data more conclusively, instead of the quick measure I placed of the salary being at least 8000 for min wage and part time workers
- Attaining hours worked data. With actual hours worked, I could compare "apples to apples" by taking into account common labor hour conditions. For example, an intern shows as being on the low end of the range. This makes sense because interns often work a few months only, but in reality may make more than other full time workers if the interns had worked the whole year at their salary.

<hr style="border:1px solid black"> </hr>

## 1/24 Question

This is a repeat of the 1/14 question, with the addition of one data point. Please refer to that response below for my approach.

<hr style="border:1px solid black"> </hr>

## 1/21 Question

Suppose you have the following dataset*, which is a list of 80 cereals, containing the following fields:

mfr: Manufacturer of cereal

A = American Home Food Products
            
G = General Mills
            
K = Kelloggs
            
N = Nabisco
            
P = Post
            
Q = Quaker Oats
            
R = Ralston Purina
        
type:

cold

hot

calories: calories per serving
       
protein: grams of protein per serving
        
fat: grams of fat per serving
      
sodium: milligrams of sodium
     
fiber: grams of dietary fiber
        
carbs: grams of complex carbohydrates
       
sugars: grams of sugars
        
potass: milligrams of potassium
        
vitamins: vitamins and minerals - 0, 25, or 100, indicating the typical percentage of FDA recommended
        
shelf: display shelf (1, 2, or 3, counting from the floor)
        
weight: weight in ounces of one serving
        
cups: number of cups in one serving
        
rating: a rating of the cereals (Possibly from Consumer Reports?)
   
__Using this data, can you determine the best 3 independent variables that help determine cereal rating?__

### Approach
To solve this problem I will:
- import data
- drop the cereal name category, as this is different for each cereal type and
- create dummy variables for the manufacturer and type columns so they can be analyzed in a regression model
- I will then perform a linear regression. The output of this will be a model with coefficients. The variables that correspond with the largest (absolute) coefficients will be the vars that have more impact on cereal rating. To do the regression:
    - I will split the data into independent vars, X, and the dependent var y.
    - I will create a train and test set of data to help validate the model
    - I will create a linear regression model and fit it to the train data
    - I will check the r2_score to determine how well the regression is fitting the trend
    - I will extract the top absolute coefficients from the model, which will inform me of the top variables that positively and negatively impacted the total rating determination


In [119]:
import pandas as pd

In [121]:
#import and shown data
cereal_df = cereal_data
cereal_df.head()


,name,mfr,type,calories,protein,fat,sodium,fiber,carbs,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


In [139]:
#drop cereal name from df
cereal_df=cereal_df.drop("name",axis=1)

In [140]:
#encode df dummies so they can be analyzed by regression model
cereal_df_encoded = pd.get_dummies(cereal_df, drop_first=True)

In [141]:
#view encoded df head
cereal_df_encoded.head()

,calories,protein,fat,sodium,fiber,carbs,sugars,potass,vitamins,shelf,weight,cups,rating,mfr_G,mfr_K,mfr_N,mfr_P,mfr_Q,mfr_R,type_H
0,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973,0,0,1,0,0,0,0
1,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679,0,0,0,0,1,0,0
2,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505,0,1,0,0,0,0,0
3,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912,0,1,0,0,0,0,0
4,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843,0,0,0,0,0,1,0


In [142]:
#create X df with independent vars
X = cereal_df_encoded.drop("rating",axis=1)

In [143]:
#create y df with dependent var
y=cereal_df_encoded["rating"]

In [144]:
#import regression and metrics libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [145]:
#split data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [146]:
#create linear regression object
lm = LinearRegression()

In [147]:
#fit train data to linear model
lm.fit(X_train,y_train)

LinearRegression()

In [148]:
#use model to predict on test data
y_pred = lm.predict(X_test)

In [149]:
#evaluate r2 score of the predictions
r2_score = r2_score(y_test,y_pred)

In [150]:
print(r2_score)

0.9999999999999994


In [167]:
#get coefficients from the model, and variable names so that top variables can be determined.
#coefficients and absolute values are obtained so results can be taken into context of which vars had most impact, and whether impact is positive or negative
coefficients= pd.DataFrame(lm.coef_)
coefficients.rename(columns={0:'coefficients'}, inplace=True)

absolute_coefficients= pd.DataFrame(lm.coef_).abs()
absolute_coefficients.rename(columns={0:'absolute_coefficients'}, inplace=True)

input_variable_names = pd.DataFrame(X_train.columns)
input_variable_names.rename(columns={0:'variable'}, inplace=True)

In [163]:
#combine above three variables into a final result table
summary_stats = pd.concat([input_variable_names,absolute_coefficients, coefficients],axis=1).sort_values(by="absolute_coefficients", ascending=False)

In [164]:
#display top 3 results showing most impactful variables on overall rating
summary_stats[:3]

,variable,absolute_coefficients,coefficients
4,fiber,3.443480,3.443480
1,protein,3.273174,3.273174
2,fat,1.691408,-1.691408


### Results

__As shown in the above table, the top three independent vars that determine cereal rating are fiber, protein and fat. Note that fiber and protein had positive coefficients and fat had a negative coefficient. This indicates having more fiber and protein helped increase rating the most, while higher fat led to the greatest decrease in a rating.__

<hr style="border:1px solid black"> </hr>

### 1/17 Question

Given the following dataset, can you write a SQL query that returns the top 3 performing ad groups each day?
    
Here we'll define performance as the ratio between revenue and spend (e.g. revenue / spend). In other words, the higher the ratio the better the peformance. The output of the query will be the date and an array of the ad groups.

### Approach

To solve the problem, I will create a query that calculates the desired ratio, ranks groups by date and displays the result as such:
- join the two datasets shown below based on date and ad_group
- within a subquery
    - calculate revenue/spend ratio as a decimal
    - use RANK to rank rows based on date and revenue/spend ratio
    - select the ratio, rank, ad_group and date within the subquery
- after the subquery
    - limit results where rank is less than/equal to 3
    - order results by date and rank for best readability
    - select date, ad_group, revenue/spend ratio, and rank in the final result

In [47]:
import pandas as pd

In [28]:
ad_table = ad_table_data
ad_table.head()

,date,shown,clicked,converted,avg_cost_per_click,total_revenue,ad
0,10/1/15,65877,2339,43,0.90,641.62,ad_group_1
1,10/2/15,65100,2498,38,0.94,756.37,ad_group_1
2,10/3/15,70658,2313,49,0.86,970.90,ad_group_1
3,10/4/15,69809,2833,51,1.01,907.39,ad_group_1
4,10/5/15,68186,2696,41,1.00,879.45,ad_group_1


In [29]:
spend_table = spend_table_data
spend_table.head()

,date,ad,total_spend
0,10/1/15,ad_group_1,2105.10
1,10/2/15,ad_group_1,2348.12
2,10/3/15,ad_group_1,1989.18
3,10/4/15,ad_group_1,2861.33
4,10/5/15,ad_group_1,2696.00


SELECT date,ad_group, revenue_spend_ratio, ratio_rank FROM (

SELECT a.date AS date, a.ad AS ad_group, round(a.total_revenue/b.total_spend, 2) AS revenue_spend_ratio, RANK () OVER ( PARTITION BY a.date ORDER BY round(a.total_revenue/b.total_spend, 2) DESC) AS ratio_rank 
FROM ad_table a JOIN spend_table b on a.ad=b.ad AND a.date=b.date)

WHERE ratio_rank<=3
order by date, ratio_rank

<hr style="border:1px solid black"> </hr>

## 1/14/2022 Question

Given the table below, called 'orders', write code to show the average revenue by month by channel:
    

|order_id |	channel |	date |	month |	revenue|
|---|---|---|---|---|
|1|	online |	2018-09-01 |	09 |	100|
|2|	online |	2018-09-03|	09 |	125|
|3|	in_store |	2018-10-11 |	10 |	200|
|4|	in_store |	2018-08-21 | 	08 |	80|
|5|	online |	2018-08-13 |	08 |	200|

Your result should return the following in a structured table:

 Month | Channel | Avg. Revenue 

### Approach

I will solve this problem two ways, based on the wording:

1. I will solve assuming that this problem is asking for revenue to be totalled for each month, and then averaged for each unique month. For example, I would calculate total revenue for January 2021 and then average that with the total revenue from January 2022, etc..

To solve the above problem I will:
- import data and pandas
- drop order_id col since it isn't needed in the final result
- group the table by month, year and channel
- aggregate the revenue col to show total revenue by month and year
- group the table by month and channel, and aggregate so an average per month can be calculated

2. I will solve assuming that this problem is asking for transactional to be totalled for each month. For example, the value of each order in September, online channel would be averaged for a final avg revenue value

To solve the above problem I will:
- import data and pandas
- drop order_id and date cols since they aren't needed in the final result
- group the table by month and channel
- aggregate the revenue col to show average revenue by transaction

In [8]:
import pandas as pd

#### Approach 1

In [9]:
#import data
channel_data_2022_01_01_approach_1 = channel_data

In [108]:
#view data
channel_data_2022_01_01_approach_1.head()

,order_id,channel,date,month,revenue
0,1,online,2018-09-01,9,100
1,2,online,2018-09-03,9,125
2,3,in_store,2018-10-11,10,200
3,4,in_store,2018-08-21,8,80
4,5,online,2018-08-13,8,200


In [109]:
#drop order_id col since it isn't relevant to analysis
channel_data_2022_01_01_approach_1.drop("order_id",axis=1, inplace=True)

In [110]:
#create a year variable based on the given date col
channel_data_2022_01_01_approach_1["year"]=pd.DatetimeIndex(channel_data_2022_01_01_approach_1['date']).year

In [111]:
#group data by month, year, channel and aggregate revenue by summing it for these combinations
revenue_by_month_and_year = channel_data_2022_01_01_approach_1.groupby(["month","year","channel"]).agg("sum")

In [112]:
#display result
revenue_by_month_and_year

revenue
month year channel          
8     2018 in_store       80
           online        200
9     2018 online        225
10    2018 in_store      200

In [126]:
#create an average revenue by month by grouping data by month and channel, and taking mean of revenue
average_revenue_by_month = revenue_by_month_and_year.groupby(["month", "channel"]).agg("mean").reset_index()

In [128]:
#rename cols
average_revenue_by_month.rename(columns={"month":"Month","channel":"Channel","revenue":"Avg. revenue"}, inplace=True)

In [129]:
#display final result
average_revenue_by_month

,Month,Channel,Avg. revenue
0,8,in_store,80
1,8,online,200
2,9,online,225
3,10,in_store,200


#### Approach 2

In [10]:
#import data
channel_data_2022_01_01_approach_2 = channel_data

In [101]:
#drop irrelevant cols
channel_data_2022_01_01_approach_2.drop("order_id",axis=1, inplace=True)
channel_data_2022_01_01_approach_2.drop("date",axis=1, inplace=True)

In [130]:
#group data by month and channel, and aggregate revenue by mean in these groups
revenue_by_transaction_2 = channel_data_2022_01_01_approach_2.groupby(["month","channel"]).agg("mean").reset_index()

In [131]:
#rename cols
revenue_by_transaction_2.rename(columns={"month":"Month","channel":"Channel","revenue":"Avg. revenue"}, inplace=True)

In [132]:
#display results
revenue_by_transaction_2

,Month,Channel,Avg. revenue
0,8,in_store,80.0
1,8,online,200.0
2,9,online,112.5
3,10,in_store,200.0


<hr style="border:1px solid black"> </hr>

## 1/12/2022 Question

Given the information below, if you had a good first interview, what is the probability you will receive a second interview?

50% of all people who received a first interview received a second interview

95% of people that received a second interview had a good first interview

75% of people that did not receive a second interview had a good first interview

### Approach

The above statistical question can be solved using Bayes' Theorem. Bayes' Theorem states that:

P(A|B) = (P(B|A) * P(A)) / P(B)

In this case:
- A is the probability of getting a second interview
- B is the probability of having a good first interview
- B|A is the probability of having a good first interview given that you received a second interview

From the above definition, A is simply the chance of getting a second interview which we are told is 50% in the first statement.

B is the probability of having a good first interview. Based on the information given, we can calculate this as the addition of the following:
- percent of people who didn't proceed to the second round (50%) but had a good first interview (75%) = 50%x75%
- percent of people of getting into the second round (50%) and having a good first interview (95%) = 50%x95%

P(B|A) is simply the 95% statistic in the second statement.

Combining this information:

In [1]:
A = .5

In [2]:
B = .5*.75 + .5*.95

In [5]:
B_given_A = .95

In [6]:
P_A_given_B = B_given_A*A / B

In [7]:
P_A_given_B

0.5588235294117647

__According to the above, there is an ~0.5588 chance that someone will receive a second interview given that you had a good first interview__

<hr style="border:1px solid black"> </hr>

## 01/07 Question

You are given a dataset with information around messages sent between users in a P2P messaging application. Below is the dataset's schema:

Given this, write code to find the fraction of messages that are sent between the same sender and receiver within five minutes (e.g. the fraction of messages that receive a response within 5 minutes). 

### Approach

To solve this problem, I will go through each row. For each row, I will:
- record current time, sender, and receiver id's
- determine if a table entry exists where sender and receiver are swapped, and time is within 5 minutes (360 seconds) of original message
- if there is such a table entry, the count of messages within 5 minutes is incremented by 1

### Result
As shown via the below code there are 86 total rows, 26 of which matched the above criteria. <strong>This leads to a fraction of .267 <strong>. 

In [1]:
#import pandas for data processing
import pandas as pd

In [93]:
#import data
message_data = pd.read_csv("https://raw.githubusercontent.com/erood/interviewqs.com_code_snippets/master/Datasets/sample_message_dataset.csv")

In [94]:
#get a quick look at data and structure
message_data.head()

,date,timestamp,sender_id,receiver_id
0,2018-03-01,1519923378,1,5
1,2018-03-01,1519942810,1,4
2,2018-03-01,1519918950,1,5
3,2018-03-01,1519930114,1,2
4,2018-03-01,1519920410,1,2


In [95]:
#count total messages in the dF
total_messages = len(message_data)
total_messages

86

In [98]:
#set counter to track instances of messages within 5 minutes and sender/reciever swapped
counter=0

#run a for loop that will take each message, search the dataframe for the time/id requirements, and increment the counter
for message_number in range(total_messages):
    time=message_data.iloc[message_number]["timestamp"]
    new_sender=message_data.iloc[message_number]["receiver_id"]
    new_reciever=message_data.iloc[message_number]["sender_id"]
    if len(message_data[(message_data["timestamp"]>time) & (message_data["timestamp"]<(time+360)) & (message_data["sender_id"]==new_sender) & (message_data["receiver_id"]==new_reciever)])>0:
        counter+=1    

In [99]:
#divide messages that met criteria by total messages to get final answer
answer = counter/total_messages
answer

0.26744186046511625

<strong>Final result is .267 <strong>

## To Solve

Suppose you're running a second-price auction. In this auction, the highest bidder will win, but will pay the auctioneer (you) the value of the second-highest bid. Assuming there are two bidders bidding on one item, and the bidder knows his own valuation but sees the valuation of the rival as uncertain and distributed uniformly in the unit interval, calculate the expected revenue when the reserve price is 1/2.
